<img width="8%" alt="Google Sheets.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Google%20Sheets.png" style="border-radius: 15%">

# Google Sheets - Update BDD Sales

**Tags:** #googlesheets #gsheet #data #naas_drivers #sales-engine #automation

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook updates "Sales" database.

## Input

### Import libraries

In [ ]:
from naas_drivers import gsheet
import pandas as pd
import os
from datetime import date
import naas_data_product
try:
    import openai
except:
    !pip install openai --user
    import openai
import time

### Setup variables
**Inputs**
- `input_dir`: Input directory to retrieve file from.
- `file_reactions`: Name of the file with reactions to be retrieved.
- `file_comments`: Name of the file with comments to be retrieved.

**Outputs**
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `append`: If False, data will be canceled and replaced.

In [ ]:
# Inputs
input_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "sales-engine", date.today().isoformat())
file_name = "sales"
openai_api_key = naas.secret.get("OPENAI_API_KEY")
spreadsheet_url = naas.secret.get("ABI_SPREADSHEET") or "YOUR_GOOGLE_SPREADSHEET_URL"
sheet_name_input = "GROWTH"
ref_contact_name = "LEADS"
ref_company_name = "LEADS_COMPANIES"

# Outputs
sheet_name_output = "SALES"
append = False

## Model

### Get data "GROWTH" from Google Sheets spreadsheet

In [ ]:
df_input = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name_input)
print("Rows:", len(df_input))
df_input.head(1)

### Get data "REF_CONTACT" from Google Sheets spreadsheet

In [ ]:
ref_contact = gsheet.connect(spreadsheet_url).get(sheet_name=ref_contact_name)
print("Ref. contact:", len(ref_contact))
ref_contact.head(1)

### Get data "REF_COMPANY" from Google Sheets spreadsheet

In [ ]:
ref_company = gsheet.connect(spreadsheet_url).get(sheet_name=ref_company_name)
print("Ref. company:", len(ref_company))
ref_company.head(1)

### Create Sales dataframe

In [ ]:
df = df_input.copy()

# Groupby
to_group = [
    "ENTITY",
    "SCENARIO",
    "LAST_INTERACTION_DATE",
    "PROFILE_URL",
]
to_agg = {
    "INTERACTION_SCORE": "sum"
}
df = df.groupby(to_group, as_index=False).agg(to_agg)
df = df.sort_values(by=["SCENARIO"], ascending=[True]).reset_index(drop=True)

# Cumsum
df["INTERACTION_CUM"] = df.groupby("PROFILE_URL", as_index=False)["INTERACTION_SCORE"].cumsum()

# Merge with Ref
ref = ref_contact[["FULLNAME", "OCCUPATION", "PROFILE_URL", "ICP", "COMPANY", "CRM_CONTACT"]]
df = pd.merge(df, ref, how="left")
df = df[(df["COMPANY"] != "TBD")]

ref2 = ref_company[["COMPANY", "COMPANY_NAME", "INDUSTRY", "STAFF_COUNT", "STAFF_RANGE", "COUNTRY"]]
df = pd.merge(df, ref2, how="left", left_on="COMPANY", right_on="COMPANY").fillna("NA")

# Filter data
df = df[
    (df["INTERACTION_CUM"] >= 3) &
    (df["COMPANY_NAME"] != "NA") &
    (~df["ICP"].isin(["NotICP", "NA"])) & 
    (df["CRM_CONTACT"].astype(str) == "False")
].drop_duplicates("PROFILE_URL")

# Cleaning
to_order = [
    "ENTITY",
    "SCENARIO",
    "LAST_INTERACTION_DATE",
    "FULLNAME",
    "OCCUPATION",
    "ICP",
    "COMPANY_NAME",
    "INDUSTRY",
    "STAFF_COUNT",
    "STAFF_RANGE",
    "COUNTRY",
    "PROFILE_URL",
    "INTERACTION_CUM",
]
to_rename = {
    "LAST_INTERACTION_DATE": "DATE",
    "INTERACTION_CUM": "INTERACTION_SCORE"
}
df = df[to_order].rename(columns=to_rename)
df = df.sort_values(by=["SCENARIO", "INTERACTION_SCORE"], ascending=[False, False]).reset_index(drop=True)

# # Prepare messaging
if not "MESSAGING_OPTIONS" in df.columns:
    df["MESSAGING_OPTIONS"] = "NA"
    
prompt_messaging = "Based on the following contact data, create 3 messaging options to engage with the person on LinkedIn, be brief, concise, casual but professional"
messagings = pload(input_dir, "messagings")
if messagings is None:
    messagings = {}
# for x in df["PROFILE_URL"].unique():
#     if x not in messagings:
#         print("Messaging for:", x)
#         contact = ref_contact[ref_contact["PROFILE_URL"] == x].reset_index(drop=True)
#         messaging = create_chat_completion(openai_api_key, prompt_messaging, str(contact.to_dict()))
#         messagings[x] = messaging
#         pdump(input_dir, messagings, "messagings")
        
df["MESSAGING_OPTIONS"] = df["PROFILE_URL"].map(messagings).fillna("NA")

# Display result
print("Rows:", len(df))
df.head(15)

## Output

### Send data to Google Sheets spreadsheet

In [ ]:
gsheet.connect(spreadsheet_url).send(sheet_name=sheet_name_output, data=df, append=append)